# Colab QLoRA Fine-Tuning Tool

This notebook walks you through training instruction-following adapters (LoRA) on top of open-source chat models such as `meta-llama/Llama-2-7b-chat-hf` using the QLoRA technique. Each section explains **what** to run and **why it matters** so you can confidently adapt the workflow to new datasets.

---

## 📚 What is QLoRA? (Quantized Low-Rank Adaptation)

**QLoRA** was introduced in the paper *"QLoRA: Efficient Finetuning of Quantized LLMs"* (Dettmers et al., 2023). It combines two powerful techniques:

1. **LoRA** (Low-Rank Adaptation) - Train small adapter matrices instead of the full model
2. **4-bit Quantization** - Compress the base model to use 75% less memory

This allows fine-tuning a **65B parameter model on a single 48GB GPU** or a **7B model on a free Google Colab T4 (16GB)**!

---

## 🧠 Understanding Quantization

### What is Quantization?

Quantization reduces the precision of model weights from 32-bit or 16-bit floats to lower bit representations.

```
Original (FP16):     [0.00012, -0.00234, 0.00567, ...]  → 16 bits per value
Quantized (4-bit):   [2, -5, 11, ...]                   → 4 bits per value
                                                         (75% memory savings!)
```

### 🔢 Types of Quantization

| Type | Bits | Memory (7B model) | Quality |
|------|------|-------------------|---------|
| FP32 | 32 | ~28 GB | Best |
| FP16/BF16 | 16 | ~14 GB | Excellent |
| INT8 | 8 | ~7 GB | Very Good |
| **NF4 (QLoRA)** | **4** | **~3.5 GB** | **Good** |

### 🌟 NF4: Normal Float 4-bit

QLoRA uses a special 4-bit format called **NF4 (Normal Float 4-bit)**:

> NF4 is information-theoretically optimal for normally distributed weights.

Neural network weights typically follow a normal (Gaussian) distribution centered around zero. NF4 places quantization levels optimally for this distribution:

```
Standard INT4:     Uniform spacing: -8, -7, -6, ... 0, ... 6, 7
NF4:               Optimal spacing:  More levels near 0 (where most weights are)
                   
                   ████████████████████████████
                ████                        ████
              ██                                ██
            ██                                    ██
          ██                                        ██
        ██                                            ██
      ──┬──┬──┬──┬─┬┬┬┬┬┬┬┬┬┬┬┬┬┬┬┬┬──┬──┬──┬──┬──
        More quantization levels near 0 (peak of distribution)
```

### 🔄 Double Quantization

QLoRA also uses **double quantization** to further reduce memory:

1. **First quantization**: Compress weights to 4-bit with scaling factors
2. **Second quantization**: Compress the scaling factors themselves to 8-bit

This saves an additional ~0.4 bits per parameter!

---

## 🔧 How QLoRA Works

```
┌────────────────────────────────────────────────────────────────────┐
│                         QLoRA ARCHITECTURE                         │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  Base Model (Frozen, 4-bit quantized)                             │
│  ┌─────────────────────────────────────────────────────────────┐  │
│  │  Attention Layer                                             │  │
│  │  ┌─────────────────┐     ┌─────────────────┐                │  │
│  │  │ q_proj (4-bit)  │ ←── │ LoRA A,B (fp16) │ ← TRAINABLE    │  │
│  │  │ k_proj (4-bit)  │ ←── │ LoRA A,B (fp16) │ ← TRAINABLE    │  │
│  │  │ v_proj (4-bit)  │ ←── │ LoRA A,B (fp16) │ ← TRAINABLE    │  │
│  │  │ o_proj (4-bit)  │ ←── │ LoRA A,B (fp16) │ ← TRAINABLE    │  │
│  │  └─────────────────┘     └─────────────────┘                │  │
│  │                                                              │  │
│  │  MLP Layer                                                   │  │
│  │  ┌─────────────────┐     ┌─────────────────┐                │  │
│  │  │ gate_proj (4-bit)│ ←── │ LoRA A,B (fp16) │ ← TRAINABLE   │  │
│  │  │ up_proj (4-bit)  │ ←── │ LoRA A,B (fp16) │ ← TRAINABLE   │  │
│  │  │ down_proj (4-bit)│ ←── │ LoRA A,B (fp16) │ ← TRAINABLE   │  │
│  │  └─────────────────┘     └─────────────────┘                │  │
│  └─────────────────────────────────────────────────────────────┘  │
│                                                                    │
│  Forward Pass:                                                     │
│  1. Dequantize 4-bit weights → BF16 on-the-fly                    │
│  2. Compute: output = W_dequant(x) + B(A(x)) × (α/r)              │
│  3. Only LoRA gradients computed (base model frozen)              │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

### 🧮 The Mathematics

For each adapted layer, the forward pass is:

$$y = W_{4bit}^{dequant} \cdot x + \frac{\alpha}{r} \cdot B \cdot A \cdot x$$

Where:
- $W_{4bit}^{dequant}$: Base weights dequantized from 4-bit to bf16 on-the-fly
- $A \in \mathbb{R}^{r \times k}$: LoRA down-projection (trainable, fp16)
- $B \in \mathbb{R}^{d \times r}$: LoRA up-projection (trainable, fp16)
- $\alpha$: Scaling factor (`lora_alpha`)
- $r$: LoRA rank

### 📊 Memory Comparison

| Setup | Model Memory | Trainable Params | Total VRAM (7B) |
|-------|--------------|------------------|-----------------|
| Full Fine-tuning (fp16) | ~14 GB | 7B | ~56 GB |
| LoRA (fp16) | ~14 GB | ~20M | ~15 GB |
| **QLoRA (4-bit + LoRA)** | **~3.5 GB** | **~20M** | **~6 GB** |

### ✅ Advantages of QLoRA

1. **Extreme memory efficiency**: Fine-tune 7B models on 8GB GPUs
2. **Near full-precision quality**: Only ~0.1% quality loss vs fp16 fine-tuning
3. **Fast inference**: 4-bit models run faster with less memory bandwidth
4. **Enables larger models**: Fine-tune 13B, 30B, 65B models on consumer hardware

### ⚠️ Trade-offs

| Aspect | QLoRA | Standard LoRA |
|--------|-------|---------------|
| Memory | Very Low | Moderate |
| Training Speed | Slightly Slower (dequantization) | Faster |
| Quality | ~99% of LoRA | Reference |
| Setup Complexity | Requires bitsandbytes | Simpler |


---

## 🖥️ Runtime Checklist

1. Go to **Runtime ▸ Change runtime type** and pick `T4 GPU` (L4/A100 if you have Colab Pro/Pro+).
2. Toggle **GPU** and keep the rest default.
3. Connect the runtime before running the cells below.

> **Why:** QLoRA loads the base model in 4-bit precision, so a T4's 16 GB VRAM is sufficient for 7B chat models when you keep batch sizes modest.

### 💾 Memory Breakdown for QLoRA

| Component | Memory (7B model) |
|-----------|-------------------|
| 4-bit quantized weights | ~3.5 GB |
| LoRA adapters (fp16) | ~50-100 MB |
| Activations & gradients | ~2-4 GB |
| Optimizer states (LoRA only) | ~200 MB |
| **Total** | **~6-8 GB** ✅ Fits on T4! |

Compare this to full fine-tuning which would need ~56 GB!


In [ ]:
!nvidia-smi


### 📦 Installing Dependencies

Key libraries for QLoRA:
- **`bitsandbytes`**: Implements 4-bit quantization (NF4) and 8-bit optimizers
- **`peft`**: Parameter-Efficient Fine-Tuning library (LoRA implementation)
- **`transformers`**: Hugging Face model loading with quantization support
- **`trl`**: Provides `SFTTrainer` for supervised fine-tuning
- **`accelerate`**: Handles device placement and mixed precision

In [ ]:
%%capture
%pip install -U accelerate==0.27.2 bitsandbytes==0.43.0 datasets==2.17.0 evaluate==0.4.1 huggingface_hub==0.21.4 peft==0.8.2 sentencepiece==0.1.99 transformers==4.38.2 trl==0.7.10 wandb


In [ ]:
from huggingface_hub import login

login(token=input("Paste your Hugging Face access token: ").strip(), add_to_git_credential=True)


---

## 📂 Data Ingestion Options

- **Upload text files** (left sidebar ▸ Files ▸ Upload) and set `DATASET_SOURCE="text_folder"`.
- **Mount Google Drive** for larger corpora:
  ```python
  from google.colab import drive
  drive.mount('/content/drive')
  ```
- **Use a public Hugging Face dataset** by supplying its repo id (e.g., `tatsu-lab/alpaca`).

The helper below consolidates these flows and creates a `datasets.Dataset` with `instruction`, `response`, and optional `system` fields.

### 🎓 Understanding Instruction-Tuning Data

Instruction-tuning teaches the model to follow human instructions. The data format:

```json
{
  "instruction": "Explain quantum computing in simple terms",
  "response": "Quantum computing uses quantum mechanics principles...",
  "system": "You are a helpful science educator."
}
```

### 📊 Config Class: QLoRA-Specific Settings

The configuration below includes parameters optimized for QLoRA training:


In [ ]:
from dataclasses import asdict, dataclass
from pathlib import Path


@dataclass
class Config:
    project_name: str = "qlora-custom-data"
    base_model: str = "meta-llama/Llama-2-7b-chat-hf"
    dataset_source: str = "text_folder"  # text_folder | hf_dataset
    text_folder: str = "/content/data"   # ignored when using hf_dataset
    hf_dataset: str | None = None         # e.g. "tatsu-lab/alpaca"
    max_samples: int | None = None
    chunk_tokens: int = 1024
    chunk_overlap: int = 128
    system_prompt: str = "You are a helpful assistant that responds with concise, domain-specific answers."
    output_dir: str = "/content/qlora-output"
    wandb_project: str | None = None

    # Training hyperparams
    micro_batch_size: int = 4
    gradient_accumulation_steps: int = 4
    epochs: float = 3.0
    learning_rate: float = 2e-4
    warmup_ratio: float = 0.03
    weight_decay: float = 0.0
    cutoff_len: int = 2048
    lora_r: int = 64
    lora_alpha: int = 16
    lora_dropout: float = 0.1
    seed: int = 42


cfg = Config()
print(asdict(cfg))


### 🔍 QLoRA Config Parameters Explained

**LoRA Hyperparameters (QLoRA uses higher rank due to quantization):**
- `lora_r=64`: Higher rank compensates for quantization information loss
- `lora_alpha=16`: Scaling factor; effective scaling is `16/64 = 0.25`
- `lora_dropout=0.1`: Slightly higher dropout for regularization

**Training Hyperparameters:**
- `micro_batch_size=4`: QLoRA uses less memory, so we can use larger batches!
- `learning_rate=2e-4`: Slightly higher LR works well with QLoRA
- `gradient_accumulation_steps=4`: Effective batch = 4 × 4 = 16

**Why higher `lora_r` for QLoRA?**
The 4-bit quantization loses some expressiveness compared to fp16. A higher LoRA rank (64 vs 16) compensates by giving the adapter more capacity to learn task-specific adjustments.

### 📈 Data Processing Pipeline

The following code handles:
1. **Text normalization**: Standardize whitespace and encoding
2. **Chunking with overlap**: Split long documents while preserving context
3. **Format unification**: Convert various formats to instruction/response pairs

In [ ]:
import json
import random
import re

import pandas as pd
from datasets import Dataset, load_dataset


def _normalize(text: str) -> str:
    text = text.replace("\r", " ").strip()
    text = re.sub(r"\s+", " ", text)
    return text


def _chunk_words(text: str, chunk_tokens: int, overlap: int) -> list[str]:
    words = text.split()
    if not words:
        return []
    step = max(chunk_tokens - overlap, 1)
    chunks = []
    for start in range(0, len(words), step):
        segment = words[start:start + chunk_tokens]
        if len(segment) < 32:
            continue
        chunks.append(" ".join(segment))
    return chunks or [" ".join(words[:chunk_tokens])]


def _load_local_texts(folder: str, cfg: Config) -> list[dict]:
    folder_path = Path(folder)
    rows = []
    for path in folder_path.rglob("*.txt"):
        text = path.read_text(encoding="utf-8")
        for chunk in _chunk_words(_normalize(text), cfg.chunk_tokens, cfg.chunk_overlap):
            rows.append({
                "instruction": f"Answer the following based on {path.stem}:",
                "response": chunk,
                "system": cfg.system_prompt,
            })
    return rows


def _load_hf_dataset(repo_id: str, cfg: Config) -> list[dict]:
    ds = load_dataset(repo_id, split="train")
    fields = set(ds.column_names)
    rows = []
    for row in ds:
        if {"instruction", "output"}.issubset(fields):
            response = row["output"]
            instruction = row["instruction"]
        else:
            # Fallback to single text field
            response = row.get("text", row.get("response", ""))
            instruction = row.get("instruction", "Summarize the passage:")
        rows.append({
            "instruction": _normalize(str(instruction)),
            "response": _normalize(str(response)),
            "system": row.get("system", cfg.system_prompt),
        })
    return rows


def build_dataset(cfg: Config) -> Dataset:
    if cfg.dataset_source == "hf_dataset" and cfg.hf_dataset:
        rows = _load_hf_dataset(cfg.hf_dataset, cfg)
    else:
        rows = _load_local_texts(cfg.text_folder, cfg)

    if cfg.max_samples:
        random.seed(cfg.seed)
        rows = random.sample(rows, min(cfg.max_samples, len(rows)))

    clean_rows = [r for r in rows if r["instruction"].strip() and r["response"].strip()]
    dataset = Dataset.from_pandas(pd.DataFrame(clean_rows))
    print(f"Dataset has {len(dataset)} rows after cleaning")
    return dataset


dataset = build_dataset(cfg)
dataset[:2]


---

## 💬 Prompt Templating

We rely on the tokenizer's built-in chat template (when available) so that formatting always matches the base model's expectations. If the checkpoint lacks a template, we fall back to a simple system/instruction/response triple.

### 🎯 Why Templates Matter for Fine-Tuning

The pre-trained model learned specific patterns during its original training. Using the correct template ensures:
1. **Consistency**: Same format during training and inference
2. **Better learning**: Model recognizes the familiar structure
3. **Proper generation**: Stop tokens and role markers work correctly

### 📝 Example Llama-2 Chat Template

```
<s>[INST] <<SYS>>
You are a helpful assistant that responds with concise, domain-specific answers.
<</SYS>>

Answer the following based on document_1: [/INST] Here is the relevant information from document_1... </s>
```


In [ ]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(cfg.base_model, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


def build_prompt(example: dict) -> dict:
    messages = [
        {"role": "system", "content": example.get("system") or cfg.system_prompt},
        {"role": "user", "content": example["instruction"]},
        {"role": "assistant", "content": example["response"]},
    ]
    if tokenizer.chat_template:
        prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    else:
        prompt = (
            f"[SYSTEM]\n{messages[0]['content']}\n\n"
            f"[USER]\n{messages[1]['content']}\n\n"
            f"[ASSISTANT]\n{messages[2]['content']}"
        )
    return {"text": prompt}


processed_dataset = dataset.map(build_prompt, remove_columns=dataset.column_names)
processed_dataset = processed_dataset.shuffle(seed=cfg.seed)
splits = processed_dataset.train_test_split(test_size=0.05, seed=cfg.seed)
splits


---

## 🚀 QLoRA Training

We load the base model in 4-bit NF4 using `BitsAndBytesConfig`, attach LoRA adapters to attention + MLP modules, and fine-tune with `trl.SFTTrainer` so padding/truncation are handled automatically.

### 🔧 BitsAndBytesConfig Deep Dive

```python
BitsAndBytesConfig(
    load_in_4bit=True,              # Enable 4-bit quantization
    bnb_4bit_quant_type="nf4",      # Use NF4 (optimal for normal distributions)
    bnb_4bit_compute_dtype=torch.bfloat16,  # Compute in bf16 for speed
    bnb_4bit_use_double_quant=True, # Quantize the quantization constants too!
)
```

**Parameter explanations:**
- `load_in_4bit`: Quantize weights to 4-bit on load
- `bnb_4bit_quant_type="nf4"`: Normal Float 4-bit, optimal for neural network weights
- `bnb_4bit_compute_dtype`: Dequantize to bf16 for matrix multiplications
- `bnb_4bit_use_double_quant`: Extra 0.4 bits/param savings by quantizing scaling factors

### 🔄 Training Flow Diagram

```
┌─────────────────────────────────────────────────────────────────────┐
│                        QLoRA TRAINING LOOP                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  1. LOAD MODEL IN 4-BIT                                            │
│     ┌─────────────────────────────────────────────────────────┐    │
│     │  for each weight matrix W:                              │    │
│     │    quantized_W = quantize_nf4(W)  # 4-bit storage      │    │
│     │    save scaling_factors           # for dequantization │    │
│     └─────────────────────────────────────────────────────────┘    │
│                                                                     │
│  2. PREPARE FOR K-BIT TRAINING                                     │
│     └─► Cast layer norms to fp32 for training stability           │
│     └─► Enable gradient checkpointing                              │
│                                                                     │
│  3. ATTACH LoRA ADAPTERS                                           │
│     ┌─────────────────────────────────────────────────────────┐    │
│     │  Target modules: q,k,v,o_proj + gate,up,down_proj       │    │
│     │  Each gets trainable A (r×k) and B (d×r) matrices      │    │
│     └─────────────────────────────────────────────────────────┘    │
│                                                                     │
│  4. FORWARD PASS (for each batch)                                  │
│     ┌─────────────────────────────────────────────────────────┐    │
│     │  W_deq = dequantize(W_4bit) → bf16  # On-the-fly       │    │
│     │  output = W_deq @ x + (B @ A @ x) × scale              │    │
│     └─────────────────────────────────────────────────────────┘    │
│                                                                     │
│  5. BACKWARD PASS                                                  │
│     └─► Gradients computed ONLY for A and B matrices              │
│     └─► Base 4-bit weights remain frozen                          │
│                                                                     │
│  6. OPTIMIZER STEP                                                 │
│     └─► Update only LoRA parameters (~0.1% of total)              │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### ⚙️ Key Training Arguments Explained

| Argument | Value | Why |
|----------|-------|-----|
| `bf16=True` | Use bfloat16 | Matches compute dtype, faster training |
| `max_grad_norm=0.3` | Gradient clipping | Prevents training instability |
| `warmup_ratio=0.03` | Gradual LR increase | Helps with 4-bit training stability |
| `packing=True` | Combine short sequences | More efficient GPU utilization |

### 🎯 Target Modules for Llama/Mistral

```python
target_modules=[
    "q_proj",    # Query projection in attention
    "k_proj",    # Key projection in attention
    "v_proj",    # Value projection in attention
    "o_proj",    # Output projection in attention
    "gate_proj", # Gating in MLP (SwiGLU)
    "up_proj",   # Up-projection in MLP
    "down_proj", # Down-projection in MLP
]
```

Applying LoRA to both attention AND MLP layers gives the best results for instruction-tuning.


In [ ]:
import math
import os

import torch
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

os.makedirs(cfg.output_dir, exist_ok=True)
if cfg.wandb_project:
    os.environ["WANDB_PROJECT"] = cfg.wandb_project
else:
    os.environ["WANDB_DISABLED"] = "true"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    cfg.base_model,
    quantization_config=bnb_config,
    device_map="auto",
)
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=cfg.lora_r,
    lora_alpha=cfg.lora_alpha,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=cfg.lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)

training_args = TrainingArguments(
    output_dir=cfg.output_dir,
    per_device_train_batch_size=cfg.micro_batch_size,
    gradient_accumulation_steps=cfg.gradient_accumulation_steps,
    num_train_epochs=cfg.epochs,
    learning_rate=cfg.learning_rate,
    bf16=True,
    logging_steps=10,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=100,
    warmup_ratio=cfg.warmup_ratio,
    weight_decay=cfg.weight_decay,
    max_grad_norm=0.3,
    report_to=([] if os.environ.get("WANDB_DISABLED") == "true" else ["wandb"]),
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    peft_config=peft_config,
    train_dataset=splits["train"],
    eval_dataset=splits["test"],
    dataset_text_field="text",
    max_seq_length=cfg.cutoff_len,
    packing=True,
    args=training_args,
)

trainer.train()
trainer.save_model(cfg.output_dir)
tokenizer.save_pretrained(cfg.output_dir)


### 📊 Model Evaluation: Perplexity

**Perplexity** is the standard metric for language models. It measures how well the model predicts the test data:

$$\text{Perplexity} = \exp\left(\frac{1}{N}\sum_{i=1}^{N} -\log P(w_i | w_{<i})\right)$$

**Interpretation:**
- Lower perplexity = better predictions
- Perplexity of 1 = perfect prediction (impossible in practice)
- Good domain-specific fine-tuning: 5-15
- Base model on specialized domain: 20-100+

### ⚠️ Note on QLoRA Evaluation
The model remains in 4-bit during evaluation. For the most accurate perplexity, you could dequantize first, but this requires more memory.

In [ ]:
from torch.utils.data import DataLoader


def compute_perplexity(eval_dataset, max_batches: int = 32) -> float:
    model.eval()
    ppl_scores = []
    loader = DataLoader(eval_dataset["text"], batch_size=1)
    for idx, batch in enumerate(loader):
        if idx >= max_batches:
            break
        encoded = tokenizer(batch[0], return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model(**encoded, labels=encoded["input_ids"])
        ppl_scores.append(math.exp(outputs.loss.item()))
    return sum(ppl_scores) / len(ppl_scores)


perplexity = compute_perplexity(splits["test"], max_batches=32)
print(f"Approximate perplexity: {perplexity:.2f}")


### 🗣️ Interactive Testing

Test your QLoRA fine-tuned model with the `chat()` function:

**Generation parameters:**
- `do_sample=True`: Enable sampling (vs greedy decoding)
- `top_p=0.9`: Nucleus sampling - only consider tokens in top 90% probability mass
- `temperature=0.7`: Controls randomness (lower = more focused, higher = more creative)

In [ ]:
def chat(prompt: str, system: str | None = None, max_new_tokens: int = 512) -> str:
    messages = [
        {"role": "system", "content": system or cfg.system_prompt},
        {"role": "user", "content": prompt},
    ]
    template = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(template, return_tensors="pt").to(model.device)
    with torch.no_grad():
        generated = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
        )
    output = generated[0][inputs["input_ids"].shape[-1]:]
    return tokenizer.decode(output, skip_special_tokens=True)


chat("Summarize the three most important facts from our dataset.")


### 🔀 Merging QLoRA Adapters

After training, you can merge the LoRA adapters into the base model for easier deployment:

**Merge process for QLoRA:**
```
1. Load QLoRA checkpoint (4-bit model + LoRA adapters)
2. Dequantize 4-bit weights → FP16
3. Merge: W_merged = W_dequant + B × A × (α/r)
4. Save as standard FP16 model
```

**Deployment options:**

| Option | Pros | Cons |
|--------|------|------|
| Keep separate (4-bit + LoRA) | Smallest size, swap adapters | Requires bitsandbytes |
| Merge to FP16 | Standard format, no dependencies | Larger size (~14 GB for 7B) |
| Re-quantize merged model | Small + standard | Slight quality loss |

**For production:** Merge and re-quantize with GPTQ/AWQ for the best balance of size and quality.

In [ ]:
from peft import AutoPeftModelForCausalLM


MERGED_DIR = Path(cfg.output_dir) / "merged"
MERGED_DIR.mkdir(parents=True, exist_ok=True)

merged_model = AutoPeftModelForCausalLM.from_pretrained(cfg.output_dir, device_map="auto")
merged_model = merged_model.merge_and_unload()
merged_model.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)
print("Merged checkpoint saved to", MERGED_DIR)

upload_to_hub = False  # flip to True to push
if upload_to_hub:
    repo_id = input("Target HF repo (e.g. username/project-name): ")
    merged_model.push_to_hub(repo_id, private=True)
    tokenizer.push_to_hub(repo_id, private=True)


---

## 🎯 Next Steps & Best Practices

### 📝 Workflow Recommendations
- Duplicate this notebook per project so you can version-control configs
- Replace the evaluation prompts with task-specific checklists (safety, compliance, tone)
- Monitor training with W&B for loss curves and gradient norms

### 🚀 Deployment Options

| Framework | QLoRA Support | Best For |
|-----------|---------------|----------|
| **vLLM** | ✅ Native | High-throughput serving |
| **TGI** | ✅ Native | Production APIs |
| **llama.cpp** | Merge first | Edge/CPU deployment |
| **Ollama** | Merge + GGUF | Local development |

### 🔧 Troubleshooting Common Issues

| Issue | Solution |
|-------|----------|
| OOM during training | Reduce `micro_batch_size` or `cutoff_len` |
| Loss not decreasing | Increase `learning_rate` or `lora_r` |
| Overfitting | Add more `lora_dropout`, reduce epochs |
| Poor generation quality | Check prompt template matches training |

### 📚 Further Reading
- [QLoRA Paper](https://arxiv.org/abs/2305.14314) - Original research
- [LoRA Paper](https://arxiv.org/abs/2106.09685) - Foundation technique
- [PEFT Documentation](https://huggingface.co/docs/peft) - Implementation details
